# AIVM Tutorial: Uploading Custom Bert Tiny Models to AIVM

This tutorial demonstrates how to upload and use custom models with the AIVM (AI Virtual Machine) system, focusing on a BERT-tiny model for SMS spam detection.

## Setup and Imports


In [1]:
# Cell 1: Import necessary libraries
import time
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import aivm_client as aic # Import the Nillion-AIVM client

Connecting to proxy localhost:50050


/home/vscode/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This cell imports the required Python libraries for:
- Neural network operations (`torch`)
- Dataset handling (`torchvision`)
- Visualization (`matplotlib`)
- AIVM client interface (`aivm_client`)

## Model Upload


In [2]:
MODEL_NAME = "BertTinySentimentTwitter" # Name of the model to be used
aic.upload_bert_tiny_model("./twitter_bert_tiny.onnx", MODEL_NAME) # Upload the model to the server

This cell demonstrates how to upload a custom model to the AIVM server:
1. Define a unique name for the model
2. Upload an ONNX-format model file using the AIVM client
3. The model will be available for encrypted inference after upload

## IMDB Sentiment Classification Implementation


In [3]:
tokenized_positive_inputs = aic.tokenize("This movie was absolutely fantastic! I loved every minute of it.",)
encrypted_positive_inputs = aic.BertTinyCryptensor(tokenized_positive_inputs[0], tokenized_positive_inputs[1])

tokenized_negative_inputs = aic.tokenize("Terrible waste of time. The plot made no sense and the acting was awful.")
encrypted_negative_inputs = aic.BertTinyCryptensor(tokenized_negative_inputs[0], tokenized_negative_inputs[1])

This cell prepares the input text for spam detection:
1. Tokenizes example messages using BERT tokenization
2. Creates an encrypted tensor from the tokenized input
3. The second message is a typical spam SMS example

## Encrypted Inference


In [4]:
%%time
result_positive = aic.get_prediction(encrypted_positive_inputs, MODEL_NAME)
result_negative = aic.get_prediction(encrypted_negative_inputs, MODEL_NAME)

print("Positive review prediction: ", result_positive)
print("Negative review prediction: ", result_negative)

Positive review prediction:  tensor([[-1.9535, -0.9669,  3.6545]])
Negative review prediction:  tensor([[ 2.4873, -0.4958, -1.9657]])
CPU times: user 14.6 ms, sys: 11.3 ms, total: 25.9 ms
Wall time: 6.65 s


This cell:
1. Performs encrypted inference using our uploaded custom model
2. Measures the execution time
3. Returns the raw model output (logits)

Output shows:
- CPU usage statistics
- Wall time for inference
- Tensor with prediction scores similar to `tensor([[-0.9099,  0.9210]])`



## Result Interpretation

In [5]:
sentiment = lambda x: "Negative" if torch.argmax(x) == 0 else "Positive" if torch.argmax(x) == 2 else "Neutral"
print("Positive review prediction: ", sentiment(result_positive))
print("Negative review prediction: ", sentiment(result_negative))

Positive review prediction:  Negative
Negative review prediction:  Positive


This cell interprets the model's prediction:
- Converts numerical output to a binary classification
- Tells whether a review is positive or negative

## Key Concepts

1. **Model Management**
   - Custom models can be uploaded to AIVM in ONNX format
   - Models are referenced by unique names
   - Uploaded models persist on the server for future use

2. **Privacy-Preserving Inference**
   - All inference is performed on encrypted data
   - Model weights and architecture remain secure
   - Results are only decrypted at the client side

3. **Performance Considerations**
   - Encrypted computation introduces some overhead
   - Wall time measurements help evaluate real-world performance
   - System is optimized for practical use cases

4. **Integration Workflow**
   1. Upload model
   2. Prepare and encrypt inputs
   3. Perform inference
   4. Interpret results
